In [150]:
import gensim.downloader as api
import numpy as np

# Load pre-trained GloVe embeddings
glove_vectors = api.load("glove-wiki-gigaword-50")  # Using 50-dimensional embeddings

def extract_vocab_and_embeddings(glove_vectors):
    vocab = {}
    embedding_matrix = []
    for idx, word in enumerate(glove_vectors.index_to_key):
        vocab[word] = idx
        embedding_matrix.append(glove_vectors[word])
    embedding_matrix = np.array(embedding_matrix)
    return vocab, embedding_matrix

vocab, embedding_matrix = extract_vocab_and_embeddings(glove_vectors)


In [151]:
# display(vocab)
display(embedding_matrix.shape)

(400000, 50)

In [152]:
display(len(vocab))

400000

In [153]:
import nltk
from nltk.stem import PorterStemmer

# Initialize the Porter Stemmer
porter = PorterStemmer()

# Stem the vocabulary
stemmed_vocab = {porter.stem(word): idx for word, idx in vocab.items()}

# Example usage:
print(len(stemmed_vocab))


332470


In [154]:
import re
def correct_duplicate(word):
    return re.sub(r'(.)\1{2,}', r'\1\1', word)

def correct_duplicate_even_more(word):
    return re.sub(r'(.)\1+', r'\1', word)

In [234]:
from nltk.stem import PorterStemmer
import nltk
import preprocessor as p
import string
import re
import contractions
from nltk.corpus import stopwords

vocab_keys = vocab.keys()

stop_words = set(stopwords.words('english'))
for word in ['not', 'no', 'nor']:
    stop_words.remove(word)

def clean_tweet(row):
    text = row['text']
    p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.HASHTAG)
    clean_text = p.clean(text)
    lower_text = clean_text.lower()
    
    expanded_text = contractions.fix(lower_text)
    fixed_text = expanded_text.replace('[^\w\s]',' ').replace('\s\s+', ' ').translate(str.maketrans('', '', string.punctuation))
    return fixed_text

def tokenize_tweet(text):
    tokens = p.tokenize(text).split()
    tokens = [correct_duplicate(token) for token in tokens if token not in stop_words]
    # print(tokens)
    # tokens = [spell.correction(token) if token not in vocab_keys and token not in stop_words else token for token in tokens]
    # print(tokens)
    # st = PorterStemmer()
    # tokens = [st.stem(word) for word in tokens]
    # filtered_tokens = [w for w in tokens if not w in stop_words]
    return tokens

In [235]:
import pandas as pd

df = pd.read_csv('data/train.csv')
df['cleaned_text'] = df.apply(clean_tweet, axis=1)
df['tokens'] = df['cleaned_text'].apply(tokenize_tweet)

In [236]:
all_tokens = set([token for tokens in df['tokens'] for token in tokens])
print(len(all_tokens))
unknown_tokens = all_tokens - set(vocab_keys)
print('Unknown tokens:', len(unknown_tokens))
print(unknown_tokens)

25013
Unknown tokens: 7843
{'marytyphoid', 'waraw', 'timegettin', 'reejuvinated', 'ooaf', 'lovah', 'ofmax', 'core2duo', '3days', 'eventho', 'hicups', 'olda', 'wov', 'gamegot', 'shawneei', 'jkidding', 'showershave', '2pm4pm', 'beotch', 'phne', 'wwbilko22com', 'creamm', 'wthat', '8521589', 'laterz', 'therapuetic', 'ceddy', 'broccolli', 'mineyou', 'hahahahahahahahahahahahaha', 'peepscorsen', 'hunrgy', 'dmc4', 'fictionbut', 'seriouslyi', 'pleaase', 'finee', 'uhoh', 'tempat', 'romulin', 'puravum', 'hillsdoctors', 'frendi', 'shannt', 'hughesy', 'naww', 'funnyyou', 'israd', 'yoouu', 'redgies', 'twune', 'possiblei', 'dhumchik', 'bitmr', 'jb3d', 'supposeand', 'starbuks', 'sufka', 'shaddupp', 'xrays', 'splitup', 'discing', 'woow', 'closedfaz', 'realnot', 'camerabags', 'arlando', 'oging', 'centernot', 'alaskaso', 'hahahahahahhah', 'ecxcited', 'notthey', 'tritonlink', 'omgthank', 'gallore', 'buccz', 'realizedi', 'faketemporary', 'sounda', 'motherstobe', 'aw0910', 'exhaausted', 'heheuntil', 'wy911'

In [237]:
in_vocab = 0
not_in = []
for token in all_tokens:
    if token in vocab:
        in_vocab += 1
    else:
        not_in.append(token)
print(in_vocab)
percentage = in_vocab / len(all_tokens) * 100
print(f'{percentage:.2f}% of tokens in the dataset are in the vocabulary')
display(len(not_in))

17170
68.64% of tokens in the dataset are in the vocabulary


7843

In [238]:
token_corrections = dict()
not_corrected = 0
for i, token in enumerate(unknown_tokens):
    tmp = correct_duplicate_even_more(token)
    if tmp in vocab_keys:
        token_corrections[token] = tmp
    else:
        for i in range(1, len(token)):
            w1, w2 = token[:i], token[i:]
            if w1 in vocab_keys and w2 in vocab_keys:
                token_corrections[token] = w1 + ' ' + w2
    if token not in token_corrections:
        not_corrected += 1
                # print('Found ', token, '->', token_corrections[token])
    # if token not in token_corrections:
    #     token_corrections[token] = spell.correction(token)
    #     print('Corrected ', token, '->', token_corrections[token])

corrected_tokens = set(token_corrections.values())
# unknown_corrected_tokens = corrected_tokens - set(vocab_keys)
print('Corrected ', len(corrected_tokens), 'tokens')

Corrected  6714 tokens


In [239]:
def substitute_unknown(row):
    tokens = row['tokens']
    corrected_tokens = [token_corrections[token] if token in token_corrections else token for token in tokens]
    return ' '.join(corrected_tokens)

In [240]:
df['corrected_text'] = df.apply(substitute_unknown, axis=1)
df['corrected_tokens'] = df['corrected_text'].apply(tokenize_tweet)

In [241]:
all_tokens = set([token for tokens in df['corrected_tokens'] for token in tokens])
print(len(all_tokens))

21016


In [242]:
unknown_tokens = all_tokens - set(vocab_keys)
print('Unknown tokens:', len(unknown_tokens))
print(unknown_tokens)

Unknown tokens: 951
{'hopeflly', 'wwhennesseyblackcom', 'greastest', 'siyal8r', 'no0o0o0o', 'reejuvinated', 'tweepsland', 'firepro0f', 'spahkly', 'kalebnationcom', 'mumzys', 'lolwellhope', 'minniapolis', 'hahhahah', 'rainlalala', 'mcflyandjb', 'biknightual', 'missinmydgbigtyme', 'suprnatural', 'core2duo', 'legarmfoot', 'whoishonorsociety', 'yogulicious', 'nobobys', 'strudlesp', 'arrngmnts', 'tweetilicious', 'caturday', 'uninhabital', 'accdentt', 'wwtwitpiccom', 'xdxdxd', 'bummerienjoyed', 'beastypops', 'peoplebrowsr', 'twitterlandhouse', 'hoilday', 'lapitytopity', 'thehodgecouk', 'backgroung', 'owwiee', 'partytiime', 'whassqoodd', 'anythgin', 'apperently', 'trnds3trs', 'wwbilko22com', 'puhleezee', 'dentention', 'playingforchangecom', 'quesadiaas', 'speedracher', 'daawhh', 'hauntingxealot', 'underpriveledged', 'owwjust', 'therapuetic', 'bunny0802', '13341015518', 'larffing', 'mommiies', 'omnomlette', 'busaysjust', 'maintanance', 'hahahahahahahahahahahahaha', 't9r8am', '3jfg', 'apearance

In [243]:
in_vocab = 0
not_in = []
for token in all_tokens:
    if token in vocab:
        in_vocab += 1
    else:
        not_in.append(token)
print(in_vocab)
percentage = in_vocab / len(all_tokens) * 100
print(f'{percentage:.2f}% of tokens in the dataset are in the vocabulary')
display(len(not_in))

20065
95.47% of tokens in the dataset are in the vocabulary


951

In [219]:
from spellchecker import SpellChecker

spell = SpellChecker()

for i, token in enumerate(unknown_tokens):
    print(i)
    token_corrections[token] = spell.correction(token)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [224]:
useful_corrections = dict()
for u in unknown_tokens:
    if token_corrections[u] != None:
        useful_corrections[u] = token_corrections[u]
display(len(useful_corrections))

281

In [226]:
corrected_tokens = set(useful_corrections.values())
unknown_2 = corrected_tokens - set(vocab_keys)
print('Unknown tokens:', len(unknown_2))
display(unknown_2)

Unknown tokens: 24


{'04071991',
 '13341015518',
 '26498457',
 '368558682',
 '5529634599',
 '61726220',
 '678998212',
 '914044621160',
 'abulia',
 'antidisestablishmentarianism',
 'barfing',
 'crumping',
 'dadoes',
 'dunnocks',
 'endodontist',
 'fieldstones',
 'jayvee',
 'kickier',
 'lolloping',
 'moustachio',
 "mum's",
 'pasquil',
 'trollers',
 'yummiest'}

In [227]:
for token, correction in useful_corrections.items():
    print(f'{token} {correction}')

hopeflly hopefully
greastest greatest
reejuvinated rejuvenated
firepro0f fireproof
spahkly sparkly
mumzys mum's
suprnatural supernatural
nobobys nobody
strudlesp strudels
uninhabital uninhabited
accdentt accident
hoilday holiday
backgroung background
anythgin anything
apperently apparently
dentention detention
underpriveledged underprivileged
therapuetic therapeutic
13341015518 13341015518
larffing barfing
mommiies mommies
maintanance maintenance
t9r8am tram
3jfg jog
apearance appearance
preciate appreciate
hunrgy hungry
duudei dude
sunbathin sunbathing
sw00p sweep
liquour liquor
thristy thirsty
vwllers killers
shloshed sloshed
61726220 61726220
halarious hilarious
exhilerating exhilarating
dec0de decide
lovliest loveliest
starbuks startups
sadifying satisfying
annpyimg annoying
deplyod deploy
witdrawal withdrawal
abueltia abulia
dissapointment disappointment
appericiate appreciate
kryptik cryptic
3to7 to
opportinists opportunists
listenening listening
26498457 26498457
thougth thought

In [280]:
import torch
import random
from torch.utils.data import Dataset

EMBED_SIZE = 50
BLOCK_SIZE = 32

class SentimentDataset(Dataset):
    def __init__(self, df):
        self.data = df
        if 'sentiment' in self.data.columns:
            label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
            df['label'] = df['sentiment'].map(label_mapping)
    
    def encode_sentence(self, tokens):
        encoded = torch.tensor(
            [embedding_matrix[vocab[token]] if token in vocab_keys else np.zeros(EMBED_SIZE) for token in tokens],
            dtype=torch.float32)
        pos = [True] * len(encoded) + [False] * (BLOCK_SIZE-len(encoded))
        random.shuffle(pos)
        padded = torch.zeros((BLOCK_SIZE, EMBED_SIZE), dtype=torch.float32)
        i = 0
        for j, p in enumerate(pos):
            if p:
                padded[j] = encoded[i]
                i += 1
        return padded
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if 'encoding' not in self.data.columns:
            self.data['encoding'] = self.data['tokens'].apply(self.encode_sentence)
        encoding = self.data.iloc[idx]['encoding']
        label = self.data.iloc[idx]['label'] if 'label' in self.data.columns else None
        return encoding, label

In [288]:
df.drop(columns=['encoding'], inplace=True, errors='ignore')

In [289]:
from sklearn.model_selection import train_test_split
TRAIN_SPLIT = 0.8

train_df, val_df = train_test_split(df, train_size=TRAIN_SPLIT, random_state=42)

In [294]:
train_data = SentimentDataset(train_df)
val_data = SentimentDataset(val_df)

In [295]:
print(len(train_data))
print(len(val_data))

19785
4947


In [302]:
import torch.nn as nn
import torch

class TransformerSentimentModel(nn.Module):
    def __init__(self, num_classes, num_heads, hidden_dim, num_layers, max_seq_len):
        super(TransformerSentimentModel, self).__init__()
        self.positional_encoding = nn.Parameter(torch.zeros(1, max_seq_len, EMBED_SIZE))
        encoder_layer = nn.TransformerEncoderLayer(d_model=EMBED_SIZE,
                                                   nhead=num_heads,
                                                   dim_feedforward=hidden_dim,
                                                   dropout=0.2)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(EMBED_SIZE, num_classes)

    def forward(self, src, src_key_padding_mask=None):
        # src = self.embedding(src) + self.positional_encoding[:, :src.size(1), :]
        src = src + self.positional_encoding[:, :src.size(1), :]
        src = src.permute(1, 0, 2)  # (S, N, E) for transformer
        output = self.transformer_encoder(src, src_key_padding_mask=src_key_padding_mask)
        output = output.mean(dim=0)  # Global average pooling
        output = self.fc(output)
        return output

In [303]:
from sklearn.metrics import f1_score, confusion_matrix

torch.manual_seed(0)
device = 'cuda'

# model = SentimentClassifier(len(dataset.vocab), 128)
model = TransformerSentimentModel(num_classes=3,
                                  num_heads=5,
                                  hidden_dim=2048,
                                  num_layers=4,
                                  max_seq_len=BLOCK_SIZE)
model.to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=True)

epochs = 100
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_dataloader)
    
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    val_loss /= len(val_dataloader)
    
    f1 = f1_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)
    acc = sum([1 if p == l else 0 for p, l in zip(all_preds, all_labels)]) / len(all_preds)
    
    print(f"Epoch {epoch+1}/{epochs} => Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, F1: {f1:.4f}")
    print(f"Accuracy: {acc:.4f}")
    display(cm)

Epoch 1/100 => Train Loss: 0.9557, Validation Loss: 0.7981, F1: 0.6508
Accuracy: 0.6479


array([[ 904,  404,   98],
       [ 474, 1237,  250],
       [ 123,  393, 1064]])

Epoch 2/100 => Train Loss: 0.7783, Validation Loss: 0.7137, F1: 0.7081
Accuracy: 0.7061


array([[ 992,  319,   95],
       [ 393, 1330,  238],
       [ 101,  308, 1171]])

Epoch 3/100 => Train Loss: 0.7334, Validation Loss: 0.6924, F1: 0.7207
Accuracy: 0.7212


...

Epoch 5/100 => Train Loss: 0.6771, Validation Loss: 0.6713, F1: 0.7231
Accuracy: 0.7271


array([[ 794,  512,  100],
       [ 138, 1588,  235],
       [  32,  333, 1215]])

Epoch 6/100 => Train Loss: 0.6636, Validation Loss: 0.6887, F1: 0.7254
Accuracy: 0.7237


array([[1013,  358,   35],
       [ 318, 1500,  143],
       [ 127,  386, 1067]])

KeyboardInterrupt: 